In [1]:
import re
import os
import collections
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Activation, Dense, Embedding, Flatten, Dropout, Conv1D, MaxPooling1D

from nltk.corpus import stopwords 

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import nltk
nltk.download('stopwords')
from gensim.models.word2vec import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import tensorflow as tf

In [6]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [7]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 448 kB 63.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [8]:
from konlpy.tag import Okt
okt = Okt()

In [9]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    # TPU 출력
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 
    # TPU 분산 전략 출력. 없으면 CPU
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.110.54.98:8470
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.110.54.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.110.54.98:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [10]:
df = pd.read_csv('/content/gdrive/MyDrive/해커톤/뉴스토픽/train_data.csv')
df.head(10)

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
5,5,팔레스타인 가자지구서 16세 소년 이스라엘군 총격에 사망,4
6,6,인도 48년 만에 파키스탄 공습…테러 캠프 폭격종합2보,4
7,7,美대선 TV토론 음담패설 만회실패 트럼프…사과 대신 빌클린턴 공격해 역효과,4
8,8,푸틴 한반도 상황 진전 위한 방안 김정은 위원장과 논의,4
9,9,특검 면죄부 받은 트럼프 스캔들 보도 언론 맹공…국민의 적,4


In [11]:
df['title'] = df['title'].apply(lambda x:x.lower())#소문자로

In [12]:
df.head(10)

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,nyt 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
5,5,팔레스타인 가자지구서 16세 소년 이스라엘군 총격에 사망,4
6,6,인도 48년 만에 파키스탄 공습…테러 캠프 폭격종합2보,4
7,7,美대선 tv토론 음담패설 만회실패 트럼프…사과 대신 빌클린턴 공격해 역효과,4
8,8,푸틴 한반도 상황 진전 위한 방안 김정은 위원장과 논의,4
9,9,특검 면죄부 받은 트럼프 스캔들 보도 언론 맹공…국민의 적,4


In [13]:
def clean_text(text):
    return re.sub('[^가-힣-ㄱ-ㅎㅏ-ㅣ]', ' ', text)#알파벳 제외 공백
def d_space(text):
    return re.sub(' +', ' ', text)#공백으로 된것을 다시 공백 한칸으로

In [14]:
df['title'] = df['title'].apply(clean_text)

In [15]:
df.head(10)

,index,title,topic_idx
0,0,인천 핀란드 항공기 결항 휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다 구글 조원 들여 전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,클린턴 측근 기업 특수관계 조명 공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
5,5,팔레스타인 가자지구서 세 소년 이스라엘군 총격에 사망,4
6,6,인도 년 만에 파키스탄 공습 테러 캠프 폭격종합 보,4
7,7,대선 토론 음담패설 만회실패 트럼프 사과 대신 빌클린턴 공격해 역효과,4
8,8,푸틴 한반도 상황 진전 위한 방안 김정은 위원장과 논의,4
9,9,특검 면죄부 받은 트럼프 스캔들 보도 언론 맹공 국민의 적,4


In [16]:
df['title'] = df['title'].apply(d_space)

In [17]:
df.head(10)

,index,title,topic_idx
0,0,인천 핀란드 항공기 결항 휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다 구글 조원 들여 전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,클린턴 측근 기업 특수관계 조명 공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
5,5,팔레스타인 가자지구서 세 소년 이스라엘군 총격에 사망,4
6,6,인도 년 만에 파키스탄 공습 테러 캠프 폭격종합 보,4
7,7,대선 토론 음담패설 만회실패 트럼프 사과 대신 빌클린턴 공격해 역효과,4
8,8,푸틴 한반도 상황 진전 위한 방안 김정은 위원장과 논의,4
9,9,특검 면죄부 받은 트럼프 스캔들 보도 언론 맹공 국민의 적,4


In [18]:
df['title'] = df['title'].apply(lambda x:x.strip())#문자열의 앞에 \n과 공백 제거

In [19]:
x = df[['title']]
y = df['topic_idx']
X_train=x['title']
y_train=y

In [20]:
classes = df['topic_idx'].nunique()
classes#클래스 갯수

7

In [21]:
result = [okt.morphs(sentence) for sentence in X_train]

In [22]:
label_encoder = preprocessing.LabelEncoder()#라벨 원핫인코딩
y = label_encoder.fit_transform(y)
y_train = to_categorical(y_train)

In [23]:
max_len = 64
embedding_dim = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(result)

In [24]:
word_index = tokenizer.word_index#X_train의 나온 단어들에 대해서 토큰화가 되고 그 토큰들에 대해서 인덱스가 매겨짐을 확인
print(word_index)

{'에': 1, '종합': 2, '서': 3, '의': 4, '로': 5, '한': 6, '일': 7, '도': 8, '이': 9, '명': 10, '대통령': 11, '은': 12, '년': 13, '한국': 14, '으로': 15, '과': 16, '들': 17, '는': 18, '대': 19, '가': 20, '위': 21, '전': 22, '하는': 23, '첫': 24, '만': 25, '월': 26, '와': 27, '등': 28, '억원': 29, '적': 30, '삼성': 31, '이란': 32, '보': 33, '중': 34, '당': 35, '감독': 36, '출시': 37, '천': 38, '장': 39, '경기': 40, '만에': 41, '게시판': 42, '주': 43, '회': 44, '에서': 45, '트럼프': 46, '신간': 47, '것': 48, '까지': 49, '문': 50, '제': 51, '원': 52, '개': 53, '안': 54, '자': 55, '정부': 56, '투자': 57, '개발': 58, '분기': 59, '지': 60, '부터': 61, '서울': 62, '을': 63, '지원': 64, '고': 65, '기': 66, '시': 67, '더': 68, '화': 69, '해야': 70, '할': 71, '최고': 72, '중국': 73, '올해': 74, '해': 75, '민주': 76, '조': 77, '내': 78, '개최': 79, '차': 80, '영업': 81, '사': 82, '세계': 83, '현대': 84, '연': 85, '비': 86, '미국': 87, '공개': 88, '최': 89, '기업': 90, '기술': 91, '간': 92, '새': 93, '증권': 94, '코로나': 95, '작년': 96, '신': 97, '대표': 98, '금융': 99, '성': 100, '시장': 101, '다': 102, '게임': 103, '최대': 104, '수': 105, '억': 106

In [25]:
sequences = tokenizer.texts_to_sequences(X_train)#위에서 토큰화가 된 문장들을 시퀀스 형태로 변환
X_train = sequence.pad_sequences(sequences, maxlen = max_len)

In [28]:
X_train

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,  368, 4913, 2500, 5227, 2718, 8750], dtype=int32)

In [27]:
y_train

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [31]:
num_words = len(word_index) + 1

In [32]:
model = Sequential()

In [33]:
model.add(Embedding(num_words,embedding_dim,
                    input_length = max_len))#텍스트를 넣기 위한 층

In [34]:
model.add(Conv1D(filters = 128, 
                 kernel_size = 9, 
                 strides = 1, 
                 padding = 'same'
                ))#1D 합성곱층

In [35]:
model.add(Activation(activation='relu'))

In [36]:
model.add(Dropout(0.3))

In [37]:
model.add(Conv1D(filters = 256, 
                 kernel_size = 6, 
                 strides = 1, 
                 padding = 'same'
                ))#1D 합성곱층

In [38]:
model.add(Activation(activation='relu'))

In [39]:
model.add(Dropout(0.4))

In [40]:
model.add(Conv1D(filters = 128, 
                 kernel_size = 3, 
                 strides = 1, 
                 padding = 'same'
                ))#1D 합성곱층

In [41]:
model.add(Activation(activation='relu'))

In [42]:
model.add(Dropout(0.5))

In [43]:
model.add(keras.layers.GlobalAveragePooling1D())

In [44]:
model.add(Flatten()) #완전 연결 계층
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(classes, activation = 'softmax'))

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 64, 100)           3173800   
_________________________________________________________________
conv1d (Conv1D)              (None, 64, 128)           115328    
_________________________________________________________________
activation (Activation)      (None, 64, 128)           0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 256)           196864    
_________________________________________________________________
activation_1 (Activation)    (None, 64, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 256)          

In [46]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer = 'Adam',
              metrics = ['acc'])

In [47]:
history = model.fit(x = X_train, 
                    y = y_train, 
                    batch_size = 128, 
                    epochs = 50, 
                    verbose = 1, 
                    validation_split = 0.1)

Epoch 1/50
321/321 [==============================] - 13s 37ms/step - loss: 1.4372 - acc: 0.3987 - val_loss: 1.2494 - val_acc: 0.4799
Epoch 2/50
321/321 [==============================] - 12s 36ms/step - loss: 0.8267 - acc: 0.6986 - val_loss: 0.9835 - val_acc: 0.6761
Epoch 3/50
321/321 [==============================] - 12s 36ms/step - loss: 0.5531 - acc: 0.8160 - val_loss: 0.9269 - val_acc: 0.6969
Epoch 4/50
321/321 [==============================] - 12s 37ms/step - loss: 0.3972 - acc: 0.8704 - val_loss: 0.9973 - val_acc: 0.6932
Epoch 5/50
321/321 [==============================] - 12s 36ms/step - loss: 0.2993 - acc: 0.9019 - val_loss: 1.1254 - val_acc: 0.6960
Epoch 6/50
321/321 [==============================] - 12s 36ms/step - loss: 0.2423 - acc: 0.9197 - val_loss: 1.2463 - val_acc: 0.6792
Epoch 7/50
321/321 [==============================] - 12s 36ms/step - loss: 0.1968 - acc: 0.9348 - val_loss: 1.4567 - val_acc: 0.6837
Epoch 8/50
321/321 [==============================] - 12s 36ms